# Visual data exploration: The The Metropolitan Museum of Art NYC
This notebook provides a visual exploration of The Metropolitan Museum of Art's Open Access Collection. We will examine the acquisition timeline, departmental distribution, artist demographics, and geographical origins of the artworks.

## Data Loading and Preprocessing

## The Art of Acquisition: An Exploration through Time and Departments

### Timeline Visualization

### Heatmap of Artworks across Departments

### Stacked Area Chart for Departmental Comparison by Acquisition Date

## Beyond the Brush: Get to Know the Artists

## Histogram of Artist Lifespans

## Pie Chart of Gender Representation

## Heatmap by Nationality & Highlight Status

## Mapping Masterpieces: Geographical Distribution of the Art

### Choropleth World Map

### Filterable Map by Department & Culture

## Conclusion